In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import drive
import pandas as pd
import io
drive.mount('/content/gdrive')
from datetime import datetime


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
#if we try to lazy in google analytics csv exports,we run into trouble
df = pd.read_csv('/content/gdrive/My Drive/final.csv')

df.head()


,Case_Type,income_2018,income_2017,income_2016,income_2015,income_2014,Per_capita_income,Median_household_income,Median_family_income,Population,Number_of_households,Number_of_families,X2019_Q4,X._of_Nation,GDP_per_capita,Household_Income_by_Race18,Household_Income_by_Race_Moe18,Household_Income_by_Race17,Household_Income_by_Race_Moe17,Household_Income_by_Race16,Household_Income_by_Race_Moe16,Household_Income_by_Race15,Household_Income_by_Race_Moe15,Household_Income_by_Race14,Household_Income_by_Race_Moe14,Household_Income_by_Race13,Household_Income_by_Race_Moe13,AVG_INCOM_13.18,Adults_With_Major_Depressive_Episode15,Adults_With_Major_Depressive_Episode16,AVG_DEPRESSIVE,Estimates_of_Chronically_Homeless_Individuals.SOCIAL_NEEDS.,Diabetes_Prevalence_18,Diabetes_Prevalence17,Diabetes_Prevalence16,Diabetes_Prevalence15,Diabetes_Prevalence14,AVG_DIABETES,Population_density_rank,Population_per_mi,Land_area_per_km,Rank,Numbers,Rank.1,mi2,km2,X._High_school_graduate,High_School_gradrank,X._Bachelor.s_degree,Bachelor_rank,Advanced_degree.percent.,Advanced_rank
0,Confirmed,63426,60212,57532,54148,51075,27646,51075,62670,3970239,1535511,966250,255418,1.2,60558,63426,1056,60212,796,57532,855,54148,912,51075,490,50251,532,28440.41667,7.51,7.89,7.700,3120,0.093,0.091,0.092,0.087,0.084,0.0894,39,41,16,27,4028977,10,95988,248607.8,0.875,34,0.248,44,0.083,44
1,Deaths,63426,60212,57532,54148,51075,27646,51075,62670,3970239,1535511,966250,255418,1.2,60558,63426,1056,60212,796,57532,855,54148,912,51075,490,50251,532,28440.41667,7.51,7.89,7.700,3120,0.093,0.091,0.092,0.087,0.084,0.0894,39,41,16,27,4028977,10,95988,248607.8,0.875,34,0.248,44,0.083,44
2,Confirmed,75277,71805,67739,64500,61933,30441,61933,71015,38802500,12758648,8762059,3183251,14.6,80563,75277,317,71805,294,67739,356,64500,395,61933,238,60190,255,33608.25000,5.90,6.04,5.970,35798,0.085,0.087,0.099,0.081,0.079,0.0862,11,251,97,1,39144818,3,155959,403932.0,0.825,50,0.326,15,0.122,15
3,Confirmed,70315,68388,65599,63488,61481,32638,61481,77941,5457173,2129195,1369594,385907,1.8,68427,70315,539,68388,658,65599,606,63488,669,61481,471,60702,432,32779.00000,7.36,7.02,7.190,813,0.080,0.079,0.078,0.076,0.075,0.0776,30,68,26,21,5489594,14,79627,206233.0,0.902,21,0.281,34,0.110,22
4,Confirmed,71953,69117,65685,63909,61303,32357,61303,75405,5355866,2039592,1315283,396367,1.8,68828,71953,655,69117,780,65685,636,63909,891,61303,487,58823,808,32920.58333,7.25,7.62,7.435,2126,0.060,0.062,0.068,0.062,0.059,0.0622,37,52,20,22,5456574,8,103642,268431.5,0.911,14,0.394,3,0.146,9


In [0]:
import pandas as pd

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
#3.探索資料
#以seaborn pairplot快速掃瞄數字型資料有沒有什麼趨勢 
sns.pairplot(df,hue='final')



KeyboardInterrupt: ignored

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f5e3645a1e0> (for post_execute):


KeyboardInterrupt: ignored

Error in callback <function flush_figures at 0x7f5e361836a8> (for post_execute):


KeyboardInterrupt: ignored

In [0]:
# 4 將資料分成訓練組及測試組

from sklearn.model_selection import train_test_split

#因為Kyphosis是之後要預測的變數，所以X軸捨棄不用
X = df.drop('Case_Type',axis=1)
y = df['Case_Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=101)

In [15]:
#5.使用決策樹演算法
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [16]:
# 6.評估決策樹模型好壞
predictions = dtree.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))




              precision    recall  f1-score   support

   Confirmed       0.50      0.34      0.40     76518
      Deaths       0.49      0.65      0.56     75834

    accuracy                           0.49    152352
   macro avg       0.49      0.50      0.48    152352
weighted avg       0.49      0.49      0.48    152352

[[26154 50364]
 [26677 49157]]


In [17]:
#7.使用隨機森林與決策樹做比較
from sklearn.ensemble import RandomForestClassifier

#n_estimators代表要使用多少CART樹（CART樹為使用GINI算法的決策樹）
rfc = RandomForestClassifier(n_estimators=100)

#從訓練組資料中建立隨機森林模型
rfc.fit(X_train, y_train)

#預測測試組是否發生
rfc_pred = rfc.predict(X_test)

#利用confusion_matrix來看實際及預測的差異
print(confusion_matrix(y_test,rfc_pred))

#利用classification_report來看precision、recall、f1-score、support
print(classification_report(y_test,rfc_pred))



[[28687 47831]
 [29199 46635]]
              precision    recall  f1-score   support

   Confirmed       0.50      0.37      0.43     76518
      Deaths       0.49      0.61      0.55     75834

    accuracy                           0.49    152352
   macro avg       0.49      0.49      0.49    152352
weighted avg       0.49      0.49      0.49    152352



In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

x, y = df.iloc[:, 1:].values, df.iloc[:, 0].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)
feat_labels = df.columns[1:]
forest = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
forest.fit(x_train, y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10000,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [22]:
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(x_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))


 1) GDP_per_capita                 0.026965
 2) Household_Income_by_Race_Moe17 0.026071
 3) X._Bachelor.s_degree           0.025881
 4) Bachelor_rank                  0.025857
 5) Adults_With_Major_Depressive_Episode15 0.025728
 6) Estimates_of_Chronically_Homeless_Individuals.SOCIAL_NEEDS. 0.025373
 7) High_School_gradrank           0.024608
 8) Advanced_rank                  0.024094
 9) AVG_DEPRESSIVE                 0.023906
10) X._High_school_graduate        0.023077
11) Adults_With_Major_Depressive_Episode16 0.023032
12) Advanced_degree.percent.       0.022723
13) Population_density_rank        0.022197
14) Rank.1                         0.022058
15) Household_Income_by_Race_Moe18 0.021829
16) mi2                            0.021713
17) Household_Income_by_Race_Moe16 0.021524
18) Household_Income_by_Race_Moe15 0.021282
19) Population_per_mi              0.021247
20) Per_capita_income              0.020845
21) Land_area_per_km               0.020697
22) Number_of_households       